In [15]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils 
from nltk.tokenize import sent_tokenize
import numpy as np
import re
import pandas as pd
import string
from string import digits
from sklearn.utils import shuffle
# from google.colab import drive
# drive.mount('/content/drive')
from tensorflow.keras.layers import Dense, Concatenate, GlobalAveragePooling1D, Input, Embedding, LSTM
from attention_maker import Attention

In [ ]:
cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [ ]:
data_file = "processed_texts.txt"
with open(data_file, encoding='utf-8') as f:
    data_list = f.read().splitlines()

In [ ]:
all_text = " ".join(data_list)
all_text = single_character_remover(all_text)
lines = page_text_split(all_text, 4)

In [ ]:
sent = np.asarray(lines)

if len(sent)%2==0:
    sentTwo = sent.reshape(len(sent)//2,2)
else:
    sent = sent[:-1]
    sentTwo = sent.reshape(len(sent)//2,2)

In [ ]:
col1, col2 = create_dataset(sentTwo)
#col1 = col1.reshape(-1)
col2 = col2.reshape(-1,1)
#col2.shape
dt = np.concatenate((col1,col2), axis=1)

lines = pd.DataFrame(dt, columns=['Query', 'Prediction'])
lines.head()

,Query,Prediction
0,mobicontrol manual mobicontrol manual,サブタイトル 設定 mobicontrol 設定
1,サブタイトル 設定 mobicontrol 設定,mobicontrol 設定 順序 端末
2,mobicontrol 設定 順序 端末,サーバ 必要 設定 事項
3,サーバ 必要 設定 事項,必須 接続 プロファイル 作成
4,必須 接続 プロファイル 作成,service サーバ 間 ad


In [ ]:
collector = []
for line in lines:
    collector.append(line.split())

question = []
answer = []
for items in collector:
    # print(items)
    shifted_df = (series_to_supervised(items, 4, 4))
    for i in range(len(shifted_df)):
        saver = shifted_df.iloc[i].values.tolist()
        length = len(saver)//2
        question.append(saver[:length])
        answer.append(saver[length:]) 
    #break

collect_question = []
for items in question:
    collect_question.append(' '.join(items))
collect_answer = []
for items in answer:
    collect_answer.append(' '.join(items))    

lines = pd.DataFrame(zip(collect_question, collect_answer), columns=['Query', 'Prediction'])


In [ ]:
lines.Prediction = lines.Prediction.apply(lambda x : '<START> '+ x + ' <END>')
lines.head()
lines.to_csv('seq2seq_dataframe.csv')

In [2]:
lines = pd.read_csv('seq2seq_dataframe.csv')

In [3]:
lines.head()

,Unnamed: 0,Query,Prediction
0,0,mobicontrol manual mobicontrol manual,<START> サブタイトル 設定 mobicontrol 設定 <END>
1,1,サブタイトル 設定 mobicontrol 設定,<START> mobicontrol 設定 順序 端末 <END>
2,2,mobicontrol 設定 順序 端末,<START> サーバ 必要 設定 事項 <END>
3,3,サーバ 必要 設定 事項,<START> 必須 接続 プロファイル 作成 <END>
4,4,必須 接続 プロファイル 作成,<START> service サーバ 間 ad <END>


In [6]:
lines = lines.head()
lines

,Unnamed: 0,Query,Prediction
0,0,mobicontrol manual mobicontrol manual,<START> サブタイトル 設定 mobicontrol 設定 <END>
1,1,サブタイトル 設定 mobicontrol 設定,<START> mobicontrol 設定 順序 端末 <END>
2,2,mobicontrol 設定 順序 端末,<START> サーバ 必要 設定 事項 <END>
3,3,サーバ 必要 設定 事項,<START> 必須 接続 プロファイル 作成 <END>
4,4,必須 接続 プロファイル 作成,<START> service サーバ 間 ad <END>


In [7]:
data = " ".join(lines.Query.values.tolist() + lines.Prediction.values.tolist())


In [7]:
data[:500]

'mobicontrol manual mobicontrol manual サブタイトル 設定 mobicontrol 設定 mobicontrol 設定 順序 端末 サーバ 必要 設定 事項 必須 接続 プロファイル 作成 <START> サブタイトル 設定 mobicontrol 設定 <END> <START> mobicontrol 設定 順序 端末 <END> <START> サーバ 必要 設定 事項 <END> <START> 必須 接続 プロファイル 作成 <END> <START> service サーバ 間 ad <END>'

In [ ]:
f = open("dataset_seq2seq.txt", "w")
f.write(data)
f.close()

In [ ]:
f = open("dataset_seq2seq.txt", "r")
mydata = f.read()

In [ ]:
mydata[:100]

'mobicontrol manual mobicontrol manual manual mobicontrol manual カスタマイザー mobicontrol manual カスタマイザー タ'

In [8]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( data.split())
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 19


In [9]:
c = 0
for items in tokenizer.word_index:
    print(items)
    if c>20:
        break
    c+=1

設定
mobicontrol
start
end
サーバ
manual
サブタイトル
順序
端末
必要
事項
必須
接続
プロファイル
作成
service
間
ad


In [10]:
questions = lines.Query 
answers = lines.Prediction

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max([ len(x) for x in tokenized_answers ])
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]

padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )


(5, 4) 4
(5, 6) 6


In [11]:
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )

In [12]:
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )


(5, 6, 19)


In [17]:
import tensorflow as tf
shape = 100

encoder_inputs = Input(shape=( None , ))
encoder_embedding = Embedding( VOCAB_SIZE, shape , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = LSTM( shape , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = Input(shape=( None ,  ))
decoder_embedding = Embedding( VOCAB_SIZE, shape , mask_zero=True) (decoder_inputs)

decoder_lstm = LSTM( shape , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')




model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 100)    1900        input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 100)    1900        input_6[0][0]                    
____________________________________________________________________________________________

In [16]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=10, epochs=100) 
model.save( 'model.h5' ) 

Epoch 1/100
1/1 [==============================] - 0s 700us/step - loss: 2.9470
Epoch 2/100
1/1 [==============================] - 0s 626us/step - loss: 2.9155
Epoch 3/100
1/1 [==============================] - 0s 661us/step - loss: 2.8860
Epoch 4/100
1/1 [==============================] - 0s 723us/step - loss: 2.8487
Epoch 5/100
1/1 [==============================] - 0s 814us/step - loss: 2.7953
Epoch 6/100
1/1 [==============================] - 0s 629us/step - loss: 2.7116
Epoch 7/100
1/1 [==============================] - 0s 747us/step - loss: 2.5734
Epoch 8/100
1/1 [==============================] - 0s 715us/step - loss: 2.3796
Epoch 9/100
1/1 [==============================] - 0s 812us/step - loss: 2.2633
Epoch 10/100
1/1 [==============================] - 0s 698us/step - loss: 2.2062
Epoch 11/100
1/1 [==============================] - 0s 731us/step - loss: 2.1582
Epoch 12/100
1/1 [==============================] - 0s 683us/step - loss: 2.1128
Epoch 13/100
1/1 [===================

In [17]:
model = models.load_model('model.h5')

In [ ]:
enc_model, dec_model = make_inference_models()

states_values = enc_model.predict(str_to_tokens( input( 'Enter question : ' )))
empty_target_seq = np.zeros((1, 1 ))
empty_target_seq[0, 0] = tokenizer.word_index['start']
stop_condition = False
decoded_translation = ''
while not stop_condition :
    dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
    print('dec_outputs :', dec_outputs)
    sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
    print('sampled_word_index: ', sampled_word_index)
    
    sampled_word = None
    for word , index in tokenizer.word_index.items() :
        if sampled_word_index == index :
            decoded_translation += ' {}'.format( word )
            sampled_word = word

    if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
        stop_condition = True
    empty_target_seq = np.zeros( ( 1 , 1 ) )  
    empty_target_seq[ 0 , 0 ] = sampled_word_index
    states_values = [ h , c ] 

print( decoded_translation )

In [21]:
lines

,Unnamed: 0,Query,Prediction
0,0,mobicontrol manual mobicontrol manual,<START> サブタイトル 設定 mobicontrol 設定 <END>
1,1,サブタイトル 設定 mobicontrol 設定,<START> mobicontrol 設定 順序 端末 <END>
2,2,mobicontrol 設定 順序 端末,<START> サーバ 必要 設定 事項 <END>
3,3,サーバ 必要 設定 事項,<START> 必須 接続 プロファイル 作成 <END>
4,4,必須 接続 プロファイル 作成,<START> service サーバ 間 ad <END>


In [19]:
shape = 100

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = Input(shape=( shape ,))
    decoder_state_input_c = Input(shape=( shape ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model
    
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    #return tokens_list
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    df = pd.DataFrame(data)
    cols = []
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = pd.concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
       agg.dropna(inplace=True)
    return agg

def page_text_split(page_text, word_limit):
    page_text = page_text.split()
    chunks = [' '.join(page_text[i:i + word_limit]) for i in range(0,
                                                                   len(page_text), word_limit)]
    return chunks


def single_character_remover(text):
    collector = []
    for items in text.split():
        if len(items) < 2:
            replaced = re.sub(r'[ぁ-んァ-ン]', '', items)
            replaced = re.sub(r'[A-Za-z]', '', replaced)
            replaced = re.sub(r'[0-9]', '', replaced)
            collector.append(replaced)
        else:
            collector.append(items)

    return ' '.join([temp.strip(' ') for temp in collector])

def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)